In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import multiprocessing
import re
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
from pytorch_tabnet.tab_model import TabNetClassifier
from joblib import dump, load

In [2]:
def get_device():
    if torch.cuda.is_available():
        device = 'cuda'
        print("CUDA 可用，使用 GPU 进行训练。")
    else:
        device = 'cpu'
        print("CUDA 不可用，使用 CPU 进行训练。")
    return device

def get_optimal_num_workers():
    try:
        num_cores = multiprocessing.cpu_count()
    except NotImplementedError:
        num_cores = 1  # 如果无法检测CPU核心数，默认设置为1
    num_workers = max(1, num_cores // 2)
    print(f"检测到的CPU核心数为{num_cores}，设置使用核心数为{num_workers}")
    return num_workers

def clean_feature_names(X):
    # 函数用于清理特征名称
    def clean_name(name):
        # 移除或替换特殊字符
        name = re.sub(r'[^\w\s-]', '_', name)
        # 确保名称不以数字开头
        if name and name[0].isdigit():
            name = 'f_' + name
        return name

    X.columns = [clean_name(col) for col in X.columns]
    return X

# 定义函数以自动检测和处理类别特征
def process_categorical_features(df, max_unique=10):
    """
    自动检测和处理数据框中的类别变量。

    参数：
    - df (pd.DataFrame): 输入的数据框。
    - max_unique (int): 判定为类别变量的最大唯一值数量。

    返回：
    - cat_idxs (list of int): 类别特征的索引。
    - cat_dims (list of int): 每个类别特征的模态数。
    - df (pd.DataFrame): 经过编码后的数据框。
    """
    cat_cols = [col for col in df.columns if df[col].nunique() <= max_unique]
    cat_dims = []
    cat_idxs = []
    encoder_dict = {}

    for col in cat_cols:
        print(f"处理类别特征: {col}，唯一值数量: {df[col].nunique()}")
        # 使用 LabelEncoder
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str).fillna('NaN'))
        cat_dims.append(len(le.classes_))
        cat_idxs.append(df.columns.get_loc(col))

    return cat_idxs, cat_dims, df

# 定义多分类加权交叉熵损失函数
class WeightedCrossEntropyLoss(nn.Module):
    def __init__(self, class_weights=None):
        """
        初始化加权交叉熵损失函数。

        参数：
        - class_weights (list或np.array或torch.Tensor): 每个类别的权重。
        """
        super(WeightedCrossEntropyLoss, self).__init__()
        if class_weights is not None:
            # 使用 register_buffer 确保 class_weights 在模型保存和加载时被包含
            self.register_buffer('class_weights', torch.tensor(class_weights, dtype=torch.float))
        else:
            self.class_weights = None

    def forward(self, y_pred, y_true):
        """
        前向传播计算损失。

        参数：
        - y_pred (torch.Tensor): 模型的预测输出，形状为 (batch_size, num_classes)。
        - y_true (torch.Tensor): 真实的标签，形状为 (batch_size,)。
        """
        if self.class_weights is not None:
            # 确保 class_weights 在 y_pred 的设备上
            return F.cross_entropy(y_pred, y_true, weight=self.class_weights.to(y_pred.device))
        else:
            return F.cross_entropy(y_pred, y_true)

device = get_device()
num_workers = get_optimal_num_workers()

CUDA 可用，使用 GPU 进行训练。
检测到的CPU核心数为64，设置使用核心数为32


In [3]:
# 读取数据
X_y_group_train = pd.read_csv('/hy-tmp/mid_data/X_y_group_train_updated_v12.2_piecewise.csv')

print("Adding numeric labels y")
le = LabelEncoder()
X_y_group_train["y"] = le.fit_transform(X_y_group_train["label"])
# 重新排列列
X_y_group_train = X_y_group_train[["dataset", "variable"] + X_y_group_train.columns.drop(["dataset", "variable", "label", "y"]).tolist() + ["label", "y"]]

# 定义要删除的列
blacklist = [
    "ttest(v,X)", 
    "pvalue(ttest(v,X))<=0.05", 
    "ttest(v,Y)", 
    "pvalue(ttest(v,Y))<=0.05", 
    "ttest(X,Y)", 
    "pvalue(ttest(X,Y))<=0.05",
    "square_dimension", 
    "max(PPS(v,others))"
]
columns_to_drop = [col for col in blacklist if col in X_y_group_train.columns]
X_y_group_train = X_y_group_train.drop(columns=columns_to_drop)

# 处理数值列的缺失值
numeric_columns = X_y_group_train.select_dtypes(include=[np.number]).columns
X_y_group_train[numeric_columns] = X_y_group_train[numeric_columns].fillna(X_y_group_train[numeric_columns].mean())

# 清理特征名称
X_y_group_train = clean_feature_names(X_y_group_train)

print("Extracting X_train, y_train, and group")
# 分离数据集ID、特征和标签
group_train = X_y_group_train["dataset"]
X = X_y_group_train.drop(["variable", "dataset", "label", "y"], axis="columns")
y = X_y_group_train["y"]

# 处理类别特征
cat_idxs, cat_dims, X = process_categorical_features(X)
print(f"类别特征索引 (cat_idxs): {cat_idxs}")
print(f"类别特征模态数 (cat_dims): {cat_dims}")

Adding numeric labels y
Extracting X_train, y_train, and group
处理类别特征: dimension，唯一值数量: 8
处理类别特征: ExactSearch_v_X_，唯一值数量: 2
处理类别特征: ExactSearch_X_v_，唯一值数量: 2
处理类别特征: ExactSearch_v_Y_，唯一值数量: 2
处理类别特征: ExactSearch_Y_v_，唯一值数量: 2
处理类别特征: ExactSearch_X_Y_，唯一值数量: 2
处理类别特征: PC_v_X_，唯一值数量: 2
处理类别特征: PC_X_v_，唯一值数量: 2
处理类别特征: PC_v_Y_，唯一值数量: 2
处理类别特征: PC_Y_v_，唯一值数量: 2
处理类别特征: PC_X_Y_，唯一值数量: 2
处理类别特征: FCI_v_X_，唯一值数量: 4
处理类别特征: FCI_X_v_，唯一值数量: 4
处理类别特征: FCI_v_Y_，唯一值数量: 4
处理类别特征: FCI_Y_v_，唯一值数量: 4
处理类别特征: FCI_X_Y_，唯一值数量: 4
类别特征索引 (cat_idxs): [0, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60]
类别特征模态数 (cat_dims): [8, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4]


In [4]:
# 分割数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print("y_train 唯一值:", np.unique(y_train))
print("y_test 唯一值:", np.unique(y_test))

# 计算类别权重（使用每个类别的逆频率）
classes = np.unique(y_train)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weights = list(class_weights)  # 转换为列表
print(f"类别权重: {class_weights}")

y_train 唯一值: [0 1 2 3 4 5 6 7]
y_test 唯一值: [0 1 2 3 4 5 6 7]
类别权重: [1.450715663384428, 0.6722329366386002, 3.281515499425947, 1.975532209012994, 0.9026654876200101, 2.093305990918412, 0.32411040301181593, 2.930285011277425]


In [ ]:
# 初始化自定义的加权交叉熵损失函数
loss_fn = WeightedCrossEntropyLoss(class_weights=class_weights)

# 定义 TabNetClassifier
clf = TabNetClassifier(
    n_d=96,                  # 决策层的宽度（小心过拟合）
    n_a=96,                  # 注意力嵌入的宽度（一般与n_d一致）
    n_steps=5,               # 决策步骤数（3-10）
    gamma=1.5,               # 特征重用系数（值接近 1 会减少层间的特征选择相关性，1.0-2.0）
    cat_idxs=cat_idxs,       # 类别特征的索引列表
    cat_dims=cat_dims,       # 每个类别特征的模态数（即类别数量）
    cat_emb_dim=1,           # 类别特征的嵌入维度
    n_independent=2,         # 每个步骤中独立的 Gated Linear Units (GLU) 层的数量（1-5）
    n_shared=5,              # 每个步骤中共享的 GLU 层的数量（1-5）
    epsilon=1e-5,            # 防止除以零的常数
    seed=42,                 # 随机种子
    momentum=0.2,           # 批量归一化的动量参数（0.01-0.4）
    clip_value=1.8,         # 如果设置为浮点数，将梯度剪裁到该值
    lambda_sparse=1e-3,      # 额外的稀疏性损失系数，值越大，模型在特征选择上越稀疏（1e-3-1e-1）
    optimizer_fn=torch.optim.Adam,                      # 优化器
    optimizer_params=dict(lr=2e-2),                     # 优化器的参数
    scheduler_fn=torch.optim.lr_scheduler.StepLR,       # 学习率调度器
    scheduler_params=dict(step_size=50, gamma=0.9),     # 学习率调度器的参数
    mask_type='entmax',   # 特征选择的掩码类型（'sparsemax' 或 'entmax'）
    # grouped_features=None,   # 将特征分组，使模型在同一组内共享注意力。这在特征预处理生成相关或依赖特征时尤其有用，例如使用 TF-IDF 或 PCA。特征重要性在同组内将相同。
    verbose=1,               # 是否打印训练过程中的信息（0 或 1）
    device_name=device,       # 使用 GPU
)

# 训练模型
clf.fit(
    X_train=X_train.values,  # 训练集的特征矩阵（np.array）
    y_train=y_train.values,  # 训练集的目标标签（np.array，对于多分类任务，标签应为整数编码）
    eval_set=[(X_train.values, y_train.values), (X_test.values, y_test.values)],   # 验证集列表
    eval_name=['train', 'valid'],                      # 验证集的名称
    eval_metric=['accuracy', 'balanced_accuracy'],     # 评估指标列表
    max_epochs=2000,         # 最大训练轮数
    loss_fn=loss_fn,            # 自定义的加权交叉熵损失函数
    patience=10,             # 早停的耐心轮数
    batch_size=1024,         # 批量大小
    virtual_batch_size=128,  # 用于 Ghost Batch Normalization 的虚拟批次大小（应能被 batch_size 整除）
    num_workers=num_workers,           # 用于 torch.utils.data.DataLoader 的工作线程数
    drop_last=False,         # 是否在训练过程中丢弃最后一个不完整的批次
    callbacks=None,          # 回调函数列表
    compute_importance=True,   # 是否计算特征重要性
)

# 预测
y_train_pred = clf.predict(X_train.values)
y_test_pred = clf.predict(X_test.values)

# 计算平衡准确率
train_score = balanced_accuracy_score(y_train, y_train_pred)
test_score = balanced_accuracy_score(y_test, y_test_pred)

print(f"训练集平衡准确率: {train_score:.6f}")
print(f"测试集平衡准确率: {test_score:.6f}")

# 输出详细的分类报告
print("\n测试集分类报告:")
print(classification_report(y_test, y_test_pred))

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 2.15835 | train_accuracy: 0.57287 | train_balanced_accuracy: 0.43368 | valid_accuracy: 0.57599 | valid_balanced_accuracy: 0.43795 |  0:00:37s
epoch 1  | loss: 1.41943 | train_accuracy: 0.62473 | train_balanced_accuracy: 0.49883 | valid_accuracy: 0.62543 | valid_balanced_accuracy: 0.49887 |  0:01:11s
epoch 2  | loss: 1.3432  | train_accuracy: 0.61606 | train_balanced_accuracy: 0.52571 | valid_accuracy: 0.61962 | valid_balanced_accuracy: 0.52746 |  0:01:46s
epoch 3  | loss: 1.30303 | train_accuracy: 0.63883 | train_balanced_accuracy: 0.5364  | valid_accuracy: 0.63781 | valid_balanced_accuracy: 0.53622 |  0:02:21s
epoch 4  | loss: 1.27541 | train_accuracy: 0.63198 | train_balanced_accuracy: 0.54103 | valid_accuracy: 0.63127 | valid_balanced_accuracy: 0.54189 |  0:02:57s
epoch 5  | loss: 1.25565 | train_accuracy: 0.64534 | train_balanced_accuracy: 0.54284 | valid_accuracy: 0.64541 | valid_balanced_accuracy: 0.54362 |  0:03:32s
epoch 6  | loss: 1.24015 | train_accuracy: 0.6